# https://datascience.blog.wzb.eu/2017/02/16/data-mining-ocr-pdfs-using-pdftabextract-to-liberate-tabular-data-from-scanned-documents/

We will use a combination of the following tools in order to reach our goal:
    - The pdftohtml command from poppler-utils to extract the texts and scanned images from the PDF
    - pdf2xml-viewer to inspect the text boxes and the generated table grid (more on that later)
    - pdftabextract to write a script that estimates the positions of columns and rows, generates a table grid and fits the text boxes into this grid in order to extract the tabular data
    
The following steps will be performed and described in detail:
    -1.Extract the scanned page images and generate an XML with the OCR texts of the PDF with pdftohtml
    -2.View the text boxes and scanned pages with pdf2xml-viewer
    -3.Load the XML describing the pages and text boxes
    -4.Detect straight lines in the scanned pages, find out a possible page skew or rotation and fix it
    -5.Detect clusters of vertical lines for identifying the columns of a table
    -6.Find out the row positions of the table by analyzing the y-coordinates’ distribution of text boxes
    -7.Create a grid of columns and lines
    -8.Match the text boxes into the grid and hence extract the tabular data in order to export it as Excel and CSV file

In [2]:
#Extract Scanned page images and generate XML with the OCR texts of the PDF using pdftohtml 
import os 
import subprocess 

#specify input/output paths
inputpdf = "//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//PO1_V2searchable.pdf"
outputxml = "//home//kgplatformuser//python//ocr//samples//xmlconverts//PO1.xml"

#run pdftohtml
#Note:It is important that you specifiy the -hidden parameter when you're dealing with OCR-processed ("sandwich") PDFs.
#You can furthermore add the parameters -f n and -l n to set only a range of pages to be converted.
#note: use subprocess going forward subprocess.call("exit 1", shell=True)
os.system("pdftohtml -c -hidden -xml -wbt 13 "+inputpdf+" "+outputxml)

#note: -wbt 14: HAD TO CHANGE THE MAX CHAR LENGTH BETWEEN WORDS CONSIDERED TO BE PART OF THE SAME WORD IN ORDER TO GET HEADERS AND TEXT

0

In [3]:
#copy output to directory containing pdfxlm viewer 
#os.system("sudo cp "+outputxml+" "+pdfxmldir)
#os.system("sudo -S true cp "+outputxml+" "+pdfxmldir)
import shutil
outputxml = "//home//kgplatformuser//python//ocr//samples//xmlconverts//PO1.xml"
pdfxmldir  = "//home//kgplatformuser//python//ocr//pdf2xml-viewer//"
shutil.copy(outputxml, pdfxmldir)
#shutil.move(outputxml, pdfxmldir)

#os.system("echo 'Needmorevisjs17' | sudo -S true cp //home//kgplatf muser//python//ocr//samples//xmlconverts//PO5.xml //home//kgplatformuser//python//ocr//pdf2xml-viewer//PO54.xml")

Change to the directory where pdf2xml-viewer resides (where its index.html or pdf2xml-viewer.html file is). You should also copy the extracted XML file and images to this location. Now let’s start up a minimal local webserver. This can be done very easily with Python:

With Python 2.x: python -m SimpleHTTPServer 8080

Now you open your browser and go to the address http://127.0.0.1:8080. The viewer shows up and you can now enter the
file name of your file to load (it must be relative to the directory in which pdf2xml-viewer resides). In pdf2xml-viewer you will now be able to browse through the pages. In the background, you can see the scanned image page and on top of that are the text boxes that were detected during OCR:

In [5]:
#Start minimal local webserver to display the text boxes in the pdf with the viewer
#os.system("sudo cp "+outputxml+" "+pdfxmldir)
import subprocess 
import os 

xml2pdfpath = "//home//kgplatformuser//python//ocr//pdf2xml-viewer//"
#os.system("python -m SimpleHTTPServer 8080")

#this executes the server 
p = subprocess.Popen("python -m SimpleHTTPServer 8080", shell=True, cwd= xml2pdfpath)
sts = os.waitpid(p.pid, 0)[1]
sts

KeyboardInterrupt: 

3. Load the XML describing the pages and text boxes
We can now start to use pdftabextract in Python code in order to load the XML file. By now, you should have installed pdftabextract via pip in the Terminal with the command pip install pdftabextract.

Let’s define some constants first that we will need throughout the script.

In [6]:
DATAPATH = "//home//kgplatformuser//python//ocr//pdf2xml-viewer//"
OUTPUTPATH = "//home//kgplatformuser//python//ocr//pdf2xml-viewer//generated_output"
INPUT_XML = 'PO1.xml'

In [7]:
#load the the XML, parse it and have a look at the pages
import os 
from pdftabextract.common import read_xml, parse_pages

# Load the XML that was generated with pdftohtml
xmltree, xmlroot = read_xml(os.path.join(DATAPATH, INPUT_XML))

# parse it and generate a dict of pages
pages = parse_pages(xmlroot)

#view output
pages

OrderedDict([(1,
              {'height': 1262,
               'image': '//home//kgplatformuser//python//ocr//samples//xmlconverts//PO1-1_1.png',
               'number': 1,
               'texts': [{'bottom': 32.0,
                 'bottomleft': array([ 23.,  32.]),
                 'bottomright': array([ 864.,   32.]),
                 'height': 12,
                 'left': 23.0,
                 'right': 864.0,
                 'top': 20.0,
                 'topleft': array([ 23.,  20.]),
                 'topright': array([ 864.,   20.]),
                 'value': ' ',
                 'width': 841,
                 'xmlnode': <Element 'text' at 0x7ff5440e1810>},
                {'bottom': 46.0,
                 'bottomleft': array([ 27.,  46.]),
                 'bottomright': array([ 374.,   46.]),
                 'height': 12,
                 'left': 27.0,
                 'right': 374.0,
                 'top': 34.0,
                 'topleft': array([ 27.,  34.]),
          

Each page consists of an xmlnode which points to the original XML page element, a page number, the page dimensions, an image (the scanned page) and the text boxes:

#### Note:The text boxes in p['texts'] contain the coordinates and dimensions of each text box as well as the content (value) and a reference to the original XML node.

In [10]:
from pprint import pprint

p_num = 1
p = pages[p_num]

print('number', p['number'])
print('width', p['width'])
print('height', p['height'])
print('image', p['image'])
print('the first three text boxes:')


#view output 
pprint(p['texts'][:1])

('number', 1)
('width', 892)
('height', 1262)
('image', '//home//kgplatformuser//python//ocr//samples//xmlconverts//PO1-1_1.png')
the first three text boxes:
[{'bottom': 32.0,
  'bottomleft': array([ 23.,  32.]),
  'bottomright': array([ 864.,   32.]),
  'height': 12,
  'left': 23.0,
  'right': 864.0,
  'top': 20.0,
  'topleft': array([ 23.,  20.]),
  'topright': array([ 864.,   20.]),
  'value': ' ',
  'width': 841,
  'xmlnode': <Element 'text' at 0x7ff5440e1810>}]


# 4. Detect straight lines in the scanned pages, find out a possible page skew or rotation and fix it

##### We can see on the scanned page image, that columns and table headers are marked with straight lines. We can detect these in order to a) find out if and how much the page is skewed or rotated and b) later use the detected vertical lines to estimate the position of the columns in the table.

In [13]:
#To detect straight lines, we can use the image processing module (pdftabextract.imgproc) which uses OpenCV’s 
#Hough transform to detect the lines.
import numpy as np
from pdftabextract import imgproc

# get the image file of the scanned page
imgfilebasename = p['image'][:p['image'].rindex('.')]
imgfile = os.path.join(DATAPATH, p['image'])

print("page %d: detecting lines in image file '%s'..." % (p_num, imgfile))

# create an image processing object with the scanned page
iproc_obj = imgproc.ImageProc(imgfile)

# calculate the scaling of the image file in relation to the text boxes coordinate system dimensions
page_scaling_x = iproc_obj.img_w / p['width']   # scaling in X-direction
page_scaling_y = iproc_obj.img_h / p['height']  # scaling in Y-direction

# detect the lines
lines_hough = iproc_obj.detect_lines(canny_kernel_size=3, canny_low_thresh=50, canny_high_thresh=150,
                                     hough_rho_res=1,
                                     hough_theta_res=np.pi/500,
                                     hough_votes_thresh=int(round(0.2 * iproc_obj.img_w)))
print("> found %d lines" % len(lines_hough))


page 1: detecting lines in image file '//home//kgplatformuser//python//ocr//samples//xmlconverts//PO1-1_1.png'...
> found 53 lines


##### we created an ImageProc instance using the scanned page image file that is referred to in the image key of the page p. ImageProc will identify the dimensions of the image file which allows us to calculate the scaling between the image dimensions and the text boxes’ coordinate system. It is important to understand that the coordinate system in image space has a different scaling than the coordinate system used to position the text boxes. For example, the image could be scanned with a size of 1000×3000 pixels (iproc_obj.img_w by iproc_obj.img_h) while the text boxes of a page are positioned on a canvas of size 500×1500 units (p['width'] by p['height']).

## Now the most crucial step is to detect the lines and use the right parameters for the image processing step. At first, our image will be converted into a binary image with white pixels marking the edges in the image (i.e. the regions with a big change in color intensity) and black pixels showing regions of homogenous color, i.e. low change in color intensity. To achieve this, the Canny algorithm is used. The kernel size is the size of the Gaussian Filter used to smooth the image and remove pixel noise. The low and high thresholds are used for detecting “strong” and “weak” edge pixels during Hysteresis Thresholding.

In [14]:
import cv2

# helper function to save an image 
def save_image_w_lines(iproc_obj, imgfilebasename):
    img_lines = iproc_obj.draw_lines(orig_img_as_background=True)
    img_lines_file = os.path.join(OUTPUTPATH, '%s-lines-orig.png' % imgfilebasename)

    print("> saving image with detected lines to '%s'" % img_lines_file)
    cv2.imwrite(img_lines_file, img_lines)

In [15]:
save_image_w_lines(iproc_obj, imgfilebasename)

> saving image with detected lines to '//home//kgplatformuser//python//ocr//samples//xmlconverts//PO1-1_1-lines-orig.png'


In [17]:
from math import radians, degrees

from pdftabextract.common import ROTATION, SKEW_X, SKEW_Y
from pdftabextract.geom import pt
from pdftabextract.textboxes import rotate_textboxes, deskew_textboxes

# find rotation or skew
# the parameters are:
# 1. the minimum threshold in radians for a rotation to be counted as such
# 2. the maximum threshold for the difference between horizontal and vertical line rotation (to detect skew)
# 3. an optional threshold to filter out "stray" lines whose angle is too far apart from the median angle of
#    all other lines that go in the same direction (no effect here)
rot_or_skew_type, rot_or_skew_radians = iproc_obj.find_rotation_or_skew(radians(0.5),    # uses "lines_hough"
                                                                        radians(1),
                                                                        omit_on_rot_thresh=radians(0.5))

# rotate back or deskew text boxes
needs_fix = True
if rot_or_skew_type == ROTATION:
    print("> rotating back by %f°" % -degrees(rot_or_skew_radians))
    rotate_textboxes(p, -rot_or_skew_radians, pt(0, 0))
elif rot_or_skew_type in (SKEW_X, SKEW_Y):
    print("> deskewing in direction '%s' by %f°" % (rot_or_skew_type, -degrees(rot_or_skew_radians)))
    deskew_textboxes(p, -rot_or_skew_radians, rot_or_skew_type, pt(0, 0))
else:
    needs_fix = False
    print("> no page rotation / skew found")
    


> no page rotation / skew found


In [ ]:
if needs_fix:
    # rotate back or deskew detected lines
    lines_hough = iproc_obj.apply_found_rotation_or_skew(rot_or_skew_type, -rot_or_skew_radians)

    save_image_w_lines(iproc_obj, imgfilebasename + '-repaired')

In [18]:
# save repaired XML (i.e. XML with deskewed textbox positions)
output_files_basename = INPUT_XML[:INPUT_XML.rindex('.')]
repaired_xmlfile = os.path.join(OUTPUTPATH, output_files_basename + '.repaired.xml')

print("saving repaired XML file to '%s'..." % repaired_xmlfile)
xmltree.write(repaired_xmlfile)

saving repaired XML file to '//home//kgplatformuser//python//ocr//pdf2xml-viewer//generated_output/PO1.repaired.xml'...


IOError: [Errno 2] No such file or directory: '//home//kgplatformuser//python//ocr//pdf2xml-viewer//generated_output/PO1.repaired.xml'